In [1]:
# https://haystack.deepset.ai/tutorials/27_first_rag_pipeline

In [2]:
import os
os.environ['OPENAI_API_KEY']='sk-111111111111111111111111111111111111111111111111'
os.environ['OPENAI_API_BASE']='http://127.0.0.1:5000/v1'

In [3]:
import requests
model_info_url = 'http://127.0.0.1:5000/v1/internal/model/info'
resp = requests.get(model_info_url)
model = resp.json()['model_name']

print(model)

Qwen1.5-32B-Chat-GPTQ-Int4


In [4]:
from haystack.document_stores.in_memory import InMemoryDocumentStore

document_store = InMemoryDocumentStore()

In [5]:
from datasets import load_dataset
from haystack import Document

dataset = load_dataset("bilgeyucel/seven-wonders", split="train")
docs = [Document(content=doc["content"], meta=doc["meta"]) for doc in dataset]

In [6]:
dataset

Dataset({
    features: ['id', 'content', 'content_type', 'meta', 'id_hash_keys', 'score', 'embedding'],
    num_rows: 151
})

In [7]:
from haystack.components.embedders import SentenceTransformersDocumentEmbedder

doc_embedder = SentenceTransformersDocumentEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")
doc_embedder.warm_up()

In [8]:
docs_with_embeddings = doc_embedder.run(docs)
document_store.write_documents(docs_with_embeddings["documents"])

Batches:   0%|          | 0/5 [00:00<?, ?it/s]

151

In [9]:
from haystack.components.embedders import SentenceTransformersTextEmbedder

text_embedder = SentenceTransformersTextEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")


In [10]:
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever

retriever = InMemoryEmbeddingRetriever(document_store)

In [11]:
from haystack.components.builders import PromptBuilder

template = """
Given the following information, answer the question.

Context:
{% for document in documents %}
    {{ document.content }}
{% endfor %}

Question: {{question}}
Answer:
"""

prompt_builder = PromptBuilder(template=template)

In [12]:
import os
from getpass import getpass
from haystack.components.generators import OpenAIGenerator

# os.environ["OPENAI_API_KEY"] = getpass("Enter OpenAI API key: ")
generator = OpenAIGenerator(model="gpt-3.5-turbo", api_base_url=os.getenv('OPENAI_API_BASE'))


In [13]:
from haystack import Pipeline

basic_rag_pipeline = Pipeline()
# Add components to your pipeline
basic_rag_pipeline.add_component("text_embedder", text_embedder)
basic_rag_pipeline.add_component("retriever", retriever)
basic_rag_pipeline.add_component("prompt_builder", prompt_builder)
basic_rag_pipeline.add_component("llm", generator)

# Now, connect the components to each other
basic_rag_pipeline.connect("text_embedder.embedding", "retriever.query_embedding")
basic_rag_pipeline.connect("retriever", "prompt_builder.documents")
basic_rag_pipeline.connect("prompt_builder", "llm")

In [14]:
question = "What does Rhodes Statue look like?"

response = basic_rag_pipeline.run({"text_embedder": {"text": question}, "prompt_builder": {"question": question}})

print(response["llm"]["replies"][0])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

The exact appearance of the Rhodes Statue, also known as the Colossus of Rhodes, is not known in detail. However, it was a statue of the Greek sun-god Helios, constructed of bronze and standing approximately 70 cubits or 33 meters (108 feet) high. The head of the statue is believed to have had curly hair with bronze or silver spikes radiating outwards, similar to the depiction on contemporary Rhodian coins. It is thought to have been constructed with an iron frame and filled with stone blocks as it was built. The statue stood with its legs apart, but archaeological evidence and engineering analysis suggest that the statue was not built in a straddling stance over the harbor entrance, as often depicted in popular imagination. Instead, it is more likely to have been矗立在一个15米高的白色大理石基座上，靠近海港入口。


In [15]:
examples = [
    "Where is Gardens of Babylon?",
    "Why did people build Great Pyramid of Giza?",
    "What does Rhodes Statue look like?",
    "Why did people visit the Temple of Artemis?",
    "What is the importance of Colossus of Rhodes?",
    "What happened to the Tomb of Mausolus?",
    "How did Colossus of Rhodes collapse?",
]

In [16]:
for question in examples:
    response = basic_rag_pipeline.run({"text_embedder": {"text": question}, "prompt_builder": {"question": question}})
    print(question)
    print(response["llm"]["replies"][0])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Where is Gardens of Babylon?
The Hanging Gardens of Babylon, also known as the Hanging Gardens of Semiramis, were located in the ancient city of Babylon, near present-day Hillah, Babil province, in Iraq. However, the exact location within Babylon has not been definitively established, as no definitive archaeological evidence has been found, and there are different theories regarding their existence and location.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Why did people build Great Pyramid of Giza?
The Great Pyramid of Giza was built as the tomb of Fourth Dynasty pharaoh Khufu. It was constructed as part of his funerary complex to serve as a lasting monument and a place of interment for the pharaoh, in accordance with ancient Egyptian beliefs about the afterlife and the need for a secure resting place for the king's soul to ensure his continuation in the next world. The pyramid was designed to protect Khufu's body and funerary objects, and its grandeur and precision were intended to reflect the pharaoh's power and divinity.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

What is the importance of Colossus of Rhodes?
The Colossus of Rhodes was a monumental statue of the Greek sun-god Helios, and it held significant importance for several reasons:

1. Architectural Achievement: As one of the Seven Wonders of the Ancient World, the Colossus of Rhodes was a remarkable testament to ancient engineering and artistic skills. Its construction, standing at approximately 33 meters (108 feet) tall, made it the tallest statue in the ancient world.

2. Symbol of Victory and Prosperity: The statue was built to celebrate the successful defense of Rhodes against the siege led by Demetrius Poliorcetes. Its construction demonstrated the city-state's wealth, military prowess, and artistic prowess, serving as a powerful symbol of their victory and resilience.

3. Cultural Significance: The statue added to the cultural and religious identity of Rhodes, as it honored their patron deity Helios, who was associated with sunlight, fertility, and protection. It also attracted vis

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

What happened to the Tomb of Mausolus?
The Tomb of Mausolus, also known as the Mausoleum at Halicarnassus, was a magnificent structure built between 353 and 350 BC for Mausolus and his sister-wife Artemisia II of Caria. Over the centuries, it withstood attacks and remained relatively intact until a series of earthquakes from the 12th to the 15th century caused significant damage. The stones of the Mausoleum were later used by the Knights of St John of Rhodes to fortify Bodrum Castle, and by the time of the Turkish invasion in 1522, only the base was recognizable. Much of the remaining portions of the tomb were further broken up and incorporated into the castle walls. Today, only the foundation remains at the site, with some surviving sculptures housed in museums like the British Museum.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

How did Colossus of Rhodes collapse?
The Colossus of Rhodes collapsed due to an earthquake that occurred in 226 BC. The earthquake caused significant damage to large portions of Rhodes, including the harbor and commercial buildings, and the statue snapped at the knees, falling over onto land.
